In [1]:
import numpy as np
import pandas as pd

# DataSet

## Attribute Information

Atributes:<br>
```
Paragraph
```

## Pre Processing

In [2]:
data = pd.read_csv('wdbc.csv')

In [3]:
data = data.drop(axis = 1, columns = ['ID'])
data.head()

Class      1      2       3       4        5        6       7        8  \
0     M  17.99  10.38  122.80  1001.0  0.11840  0.27760  0.3001  0.14710   
1     M  20.57  17.77  132.90  1326.0  0.08474  0.07864  0.0869  0.07017   
2     M  19.69  21.25  130.00  1203.0  0.10960  0.15990  0.1974  0.12790   
3     M  11.42  20.38   77.58   386.1  0.14250  0.28390  0.2414  0.10520   
4     M  20.29  14.34  135.10  1297.0  0.10030  0.13280  0.1980  0.10430   

        9   ...        21     22      23      24      25      26      27  \
0  0.2419   ...     25.38  17.33  184.60  2019.0  0.1622  0.6656  0.7119   
1  0.1812   ...     24.99  23.41  158.80  1956.0  0.1238  0.1866  0.2416   
2  0.2069   ...     23.57  25.53  152.50  1709.0  0.1444  0.4245  0.4504   
3  0.2597   ...     14.91  26.50   98.87   567.7  0.2098  0.8663  0.6869   
4  0.1809   ...     22.54  16.67  152.20  1575.0  0.1374  0.2050  0.4000   

       28      29       30  
0  0.2654  0.4601  0.11890  
1  0.1860  0.2750  0.08902  
2  0.2430  0.3613  0.08758  
3  0.2575  0.6638  0.17300  
4  0.1625  0.2364  0.07678  

[5 rows x 31 columns]

In [4]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data.Class = encoder.fit_transform(data.Class)

In [5]:
X = data.iloc[:, 1:].values
y = data.Class.values

# Bagging & Random Subspace

In [56]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Perceptron
from sklearn.metrics import roc_auc_score
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import f1_score

In [57]:
features = [1.0, 0.5]
models = [DecisionTreeClassifier(), Perceptron(tol = 0.0001, max_iter = 10000)]
sizes = [1.0, 0.9, 0.8, 0.7, 0.6, 0.5]
metrics = np.zeros((10,6,4))

In [9]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10)
StratifiedKFold(n_splits=2, random_state=None, shuffle=False)

StratifiedKFold(n_splits=2, random_state=None, shuffle=False)

In [59]:
for feature in features:
    print(feature, "features:")
    for model in models:
        fold = 0
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            for i, size in enumerate(sizes):
                # All data points
                bg = BaggingClassifier(model,
                                         max_samples = size,
                                         max_features = feature,
                                         n_estimators = 100,
                                         random_state = 42)
                bg.fit(X_train, y_train)

                metrics[fold, i, :] = [roc_auc_score(y_test, bg.predict_proba(X_test)[:,1]),
                      bg.score(X_test, y_test),
                      geometric_mean_score(y_test, bg.predict(X_test)),
                      f1_score(y_test, bg.predict(X_test))]
            fold += 1
        print()
        print(model, metrics)


1.0 features:

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best') [[[0.99368687 0.94827586 0.94948152 0.93333333]
  [0.99368687 0.94827586 0.94948152 0.93333333]
  [0.99494949 0.94827586 0.94948152 0.93333333]
  [0.99558081 0.96551724 0.97182532 0.95652174]
  [0.99368687 0.94827586 0.94948152 0.93333333]
  [0.99558081 0.94827586 0.94948152 0.93333333]]

 [[0.95138889 0.9137931  0.90313707 0.88372093]
  [0.96843434 0.89655172 0.88975652 0.86363636]
  [0.95328283 0.87931034 0.8660254  0.8372093 ]
  [0.95075758 0.9137931  0.90313707 0.88372093]
  [0.94633838 0.89655172 0.88975652 0.86363636]
  [0.9520202  0.89655172 0.88975652 0.86363636]]

 [[0.98809524 0.94736842 0.93788572 0.92682927]
  [0.98


Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      max_iter=10000, n_iter=None, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=0.0001, verbose=0, warm_start=False) [[[0.95328283 0.9137931  0.90313707 0.88372093]
  [0.93434343 0.86206897 0.82877541 0.8       ]
  [0.92676768 0.86206897 0.82877541 0.8       ]
  [0.94760101 0.89655172 0.87904907 0.85714286]
  [0.94191919 0.87931034 0.8660254  0.8372093 ]
  [0.95833333 0.87931034 0.85428214 0.82926829]]

 [[0.93623737 0.86206897 0.81417368 0.78947368]
  [0.93939394 0.82758621 0.82572282 0.7826087 ]
  [0.93308081 0.87931034 0.8660254  0.8372093 ]
  [0.9375     0.9137931  0.87904907 0.87179487]
  [0.90214646 0.79310345 0.79772404 0.75      ]
  [0.90530303 0.77586207 0.75377836 0.69767442]]

 [[0.9728836  0.9122807  0.87287156 0.86486486]
  [0.96957672 0.87719298 0.83333333 0.81081081]
  [0.96230159 0.87719298 0.84827868 0.82051282]
  [0.96031746 0.89473684 0.88640526 0.85714286]
  [0.9702381  0.8